## Project Description: Next Word Prediction Using GRU RNN

In [1]:
# Import Libraries
import nltk
import  pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,GRU
from tensorflow.keras.layers import Input

#### Preprocessing

In [2]:
# Laod the dataset
with open('hamlet.txt','r') as file:
    text=file.read().lower()

# Tokenize the text-creating indexes for words
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
print(f"Total word is : {total_words}")

# create input sequences
input_sequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad Sequences
max_sequence_len=max([len(x) for x in input_sequences])

# Padding input sequences
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))

Total word is : 4818


In [3]:
#Create Features and Label
X,y=input_sequences[:,:-1],input_sequences[:,-1]

In [4]:
X

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]])

In [5]:
y

array([ 687,    4,   45, ..., 1047,    4,  193])

In [6]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

This line converts integer-encoded target labels (y) into one-hot encoded categorical vectors. It transforms each integer label into a binary vector where the index of the target word is marked as 1, and all other indices are 0. The num_classes parameter specifies the total number of unique words in the vocabulary. This is commonly used in neural network classification tasks to prepare target labels for training.

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Model
model = Sequential()
model.add(Input(shape=(max_sequence_len,)))  # Input layer
model.add(Embedding(input_dim=total_words,  
                    output_dim=100))      
model.add(GRU(150, return_sequences=True)) 
model.add(Dropout(0.2))                  
model.add(GRU(100))                        
model.add(Dense(total_words, activation="softmax"))  

# Compile
model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 14, 100)             │         481,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 14, 150)             │         113,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 14, 150)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 100)                 │          75,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4818)                │         486,618 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,157,418 (4.42 MB)

 Trainable params: 1,157,418 (4.42 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Train the model
history=model.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test),verbose=1,callbacks=[early_stopping])

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 22s 29ms/step - accuracy: 0.0330 - loss: 7.1689 - val_accuracy: 0.0354 - val_loss: 6.7933
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 21s 29ms/step - accuracy: 0.0413 - loss: 6.4715 - val_accuracy: 0.0532 - val_loss: 6.8288
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 22s 31ms/step - accuracy: 0.0511 - loss: 6.1828 - val_accuracy: 0.0635 - val_loss: 6.7751
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.0676 - loss: 5.9561 - val_accuracy: 0.0764 - val_loss: 6.7993
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.0836 - loss: 5.6748 - val_accuracy: 0.0727 - val_loss: 6.9505
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.0937 - loss: 5.3971 - val_accuracy: 0.0748 - val_loss: 7.0034
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.1048 - loss: 5.1587 - val_accuracy: 0.0738 - val_loss: 7.1008
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 19s 27ms/step - accuracy: 0.1204 - loss: 4.9029 - 

In [10]:
# Save the model
model.save("gru_next_word_model.h5")
# Save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)